# Initial analysis script - bioinformatic differences

This script goes through the first analysis step of identifying discrepancies between the 4 methods

1. ABRicate (our baseline method)
2. ARIBA
3. KmerResistance
4. SRST2

It essentially reads in all the results we obtained, and then matches what genes were found
Following this, it defines gene clusters, and partitions set of groups into its gene "culster"

**I.e. The bit in the red box from the image below**

![image](method_1.png)


It then uses this output to produce **Figure X** from the paper which describes initial discrepancy for each gene family/cluster as well as identify relavent contigs for simulations

### Outputs




### Setup

**So first steps are to load in required modules and then identify all the output reports**


#### Dependencies

1. Python 3 
2. Biopython
3. Pandas
4. Numpy
5. tqdm
6. networkx

#### Inputs
Some notes for this step, firstly which files we take
1. for ABRicate we take each contigs.tab file
2. for ARIBA we use its summary file
3. for KmerResistance we use the .KmerRes files
4. for SRST2 we use the .out__fullgenes__seqs_clustered__results.txt files

Note these were chosen as they seem to follow guidelines and where no guidelines available, give us the most closely matching results between the four programs

Note also SRST2 does not produce output if it finds no genes. The others all do

#### Resfinder database

For this we load
1. The naming link database
2. For each of the sub databases by antibiotic class 
  * beta-lactam
  * quinolone
  * aminoglycoside
  * sulfa antibiotics
  * trimethoprim
3. The whole database

We do 1 so the results are interpretable
We do 2 so we can breakdown results by antibiotic class as well as specific genes
We do 3 to do all versus all of the resistance database


In [1]:
# We start by importing required modules
# File structure
import os
import csv


# Pandas
import pandas as pd
import numpy as np
import networkx as nx
# These are the fundamental modules used for analysing the data


# Sequence manipulation
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
# These modules are priarily used for loading the database, and then for identifying whether there are any
# Perfrect protein matches in the dataset


# Plotting


# Other general
from tqdm import tnrange, tqdm_notebook
from collections import Counter
# We just import this to check our code runs sensibly and to get timing estimates for stuff
from copy import deepcopy


In [2]:
# Loading in the output files




# Note because of the different way in which ARIBA is loaded in we also add a special reading function
def ariba_parser(s):
    s_clusters = sorted(list(set([k.split(".")[0] for k in s.index if "cluster" in k and ".match" in k])))
    s_clusters = [k for k in s_clusters if s[k+".match"] == "yes"]
    s_genes = [s[k+ ".ref_seq"] for k in s_clusters]
    return s_genes



In [3]:
# Finally loading in the resfinder database
linkfile = "../resistance_databases/resfinder_20191001_link.csv"
resfinder_fulldb = "../resistance_databases/resfinder_20191001_formatted.fasta"
blm_db = "../resistance_databases/resfinder_20191001/beta-lactam.fsa"
qui_db = "../resistance_databases/resfinder_20191001/quinolone.fsa"
ami_db = "../resistance_databases/resfinder_20191001/aminoglycoside.fsa"
sul_db = "../resistance_databases/resfinder_20191001/sulphonamide.fsa"
tri_db = "../resistance_databases/resfinder_20191001/trimethoprim.fsa"

# 1. loading the link file
resfinder_link = pd.read_csv(linkfile, index_col=0, header=None)
resfinder_rlink = pd.read_csv(linkfile, index_col=1, header=None)

# 2. loading the sub files
# Note this removes duplicate Ids, this was checked as to what it did in the database formatting files
blm_db = {k.id:k for k in SeqIO.parse(blm_db, "fasta")}
qui_db = {k.id:k for k in SeqIO.parse(qui_db, "fasta")}
ami_db = {k.id:k for k in SeqIO.parse(ami_db, "fasta")}
sul_db = {k.id:k for k in SeqIO.parse(sul_db, "fasta")}
tri_db = {k.id:k for k in SeqIO.parse(tri_db, "fasta")}

# 3. loading the whole database
res_db = SeqIO.to_dict(SeqIO.parse(resfinder_fulldb, "fasta"))

# Next were going to do all vs all similarity of the resfinder database

sim_matrix = pd.DataFrame(np.zeros((len(res_db.keys()),len(res_db.keys()) )), 
                         columns = sorted(list(res_db.keys())), index=sorted(list(res_db.keys())))
# For this similarity we will use 17-mers (one of the prefixes using MASH, note several others were checked prior to this for their effects)
# Clusters marked by this are fairly similar to other k-mer sizes
res_db_kmers = {k: set([str(res_db[k].seq)[i:i+17] for i in range(len(res_db[k].seq)-16)]) for k in res_db.keys()}

def calculate_jac_sim(l1, l2):
    intersection = len(res_db_kmers[l1].intersection(res_db_kmers[l2]))
    union = len(res_db_kmers[l1].union(res_db_kmers[l2]))
    return(intersection/union)

# This code actually goes through populating this matrix, 
# However takes 30 minutes to run, and for the sake of running this code quickly on line, I will
# use a matrix I made earlier (using thie code)
# for n in tnrange(len(res_db_kmers)):
#     k  = list(res_db_kmers.keys())[n]
#     for j in res_db_kmers:
#         sim_matrix.loc[k][j] = calculate_jac_sim(k, j)

sim_matrix = pd.read_csv("readymade_sim_matrix.csv", index_col = 0)



### Setting up the analysis class


In the next section of code, the aim is to define a class which performs most of the comparisons for us. 

#### Defining useful functions
Before we set up the class we define useful functions
**CLUSTERING**
recursive cluster => This essentially uses netrowkx to greate a graph, which links togehter elements with non-zero similarity. 
The other two functions make_tuples and name list are simplications of bits within the recursive cluster functions

**AGREEMENT**

![image](method_combinations.png)



#### Reading in the data

This does the following steps
For each of ABRicate, ARIBA, KmerResistance, SRST2 we
1. read in its file
2. Pull out the TRGs it identifies and relabel them with their original names
3. Separate these into groups according to their relavent antibiotics.



#### Comparing outputs

The next stage is to define what genes we have in the sample, here we then first
1. make a combined dict of all genes found. 
2. Then separate according to clusters
3. make identification patterns for each of these clusters



This is done using an external spreadsheet (which suggests putative families for all patterns of genes seen)

In [4]:
# First we define three useful functions we use later 


###### CLUSTERING FUNCTIONS  ######


def make_tuples(l):
    # Make all possible tuples in a list
    output = []
    for i in range(len(l)):
        for j in range(len(l)):
            output.append((l[i],l[j]))
    output = sorted(list(set(output)))
    return output

def name_list(l, d):
    #rename all elements of a list
    return [d[k] for k in l]


def recursive_cluster(df, l):
    groups = {}
    # First we get all linked pais.
    for i in l:
        i_data = df.loc[i]
        i_group = [i]
        for j in l:
            if j != i: 
                if i_data[j] != 0:
                    i_group.append(j)
        i_group = sorted(i_group)
        groups[i] =  i_group
    # Assign numbers to the elements of l and then generate a dictionary to link numbers and names
    naming = {}
    reverse_naming  = {}
    m = 1
    for i in l:
        naming[i] = m
        reverse_naming[m] = i
        m += 1
    # Grouping tuples like a graph using networkx
    final_tuples = []
    for i in groups:
        final_tuples = final_tuples + make_tuples([naming[j] for j in groups[i]])
    final_tuples = sorted(list(set(final_tuples)))
    graph=nx.Graph(final_tuples)
    output = [name_list(list(c), reverse_naming) for c in nx.connected_components(graph)]
    return output

###### AGREEMENT PATTERN FUNCTIONS 

# Note for these functions they always assume the results are put in the correct order
# i.e. ABRicate, ARIBA, KmerResistance, SRST2

# First we start with a general agreement function 
def agreement_pattern(l1, l2, l3, l4):
    args = deepcopy(locals())
    arg_list = ['l1', 'l2', 'l3', 'l4']
    for key in arg_list:
        args[key] = ":".join(sorted(args[key]))
    patterns = {}
    output = []
    starting_no = 0
    for key in arg_list:
        if args[key] not in patterns:
            starting_no += 1
            patterns[args[key]] = starting_no
            output.append(starting_no)
        else:
            output.append(patterns[args[key]])
    return output
    

# Now for gene agreement, I use this program to say which genes (from a list) each method has found
def pres_bin(l1, l2):
    output = []
    for k in l1:
        if k in l2:
            output.append("1")
        else:
            output.append("0")
    return output

# Here is the agreement function again, but this time i've dropped the sort function. 
# This enables me to use the output from pres_bin directly to make the patterns as defined above 
def pres_bin_agreement_pattern(l1, l2, l3, l4):
    args = deepcopy(locals())
    arg_list = ['l1', 'l2', 'l3', 'l4']
    for key in arg_list:
        args[key] = ":".join(args[key])
    patterns = {}
    output = []
    starting_no = 0
    for key in arg_list:
        if args[key] not in patterns:
            starting_no += 1
            patterns[args[key]] = starting_no
            output.append(starting_no)
        else:
            output.append(patterns[args[key]])
    return output
    



In [5]:

# Reading in each of the data

class isolate:
    
    def __init__(self,guuid):
        self.guuid = guuid
        # First lest start with ABRicate, for this one we take everything assuming control
        # Each section of this code does similar things, 1. read the file , 2 translate the genes, 
        # 3 separate by the classes were interested in.
        self.abricate_fl = pd.read_csv(abricate_files[self.guuid], delimiter= "\t").fillna("")
        self.abricate_fl = self.abricate_fl.loc[self.abricate_fl['%COVERAGE'] > 60.0]
        self.abricate_fl = self.abricate_fl.loc[self.abricate_fl['%IDENTITY'] > 90.0]
        self.abricate_genes = sorted(list(set([resfinder_link.loc[k][1] for k in list(self.abricate_fl['GENE'])])))
        self.abricate_trg = {"blm":sorted([k for k in self.abricate_genes if k in blm_db.keys()]),
                             "qui":sorted([k for k in self.abricate_genes if k in qui_db.keys()]), 
                             "ami":sorted([k for k in self.abricate_genes if k in ami_db.keys()]),
                             "sul":sorted([k for k in self.abricate_genes if k in sul_db.keys()]), 
                             "tri":sorted([k for k in self.abricate_genes if k in tri_db.keys()])}
        self.abricate_alltrg = sorted(list(set(self.abricate_trg['blm'] + self.abricate_trg['qui'] + 
                                               self.abricate_trg['ami'] + self.abricate_trg['sul'] +
                                              self.abricate_trg['tri'])))
        
        # Next ARIBA
        self.ariba_data = ariba_parser(ariba_summary.loc[self.guuid])
        self.ariba_genes = sorted(list(set([resfinder_link.loc[k][1] for k in self.ariba_data])))
        self.ariba_trg = {"blm":sorted([k for k in self.ariba_genes if k in blm_db.keys()]),
                             "qui":sorted([k for k in self.ariba_genes if k in qui_db.keys()]), 
                             "ami":sorted([k for k in self.ariba_genes if k in ami_db.keys()]),
                             "sul":sorted([k for k in self.ariba_genes if k in sul_db.keys()]), 
                             "tri":sorted([k for k in self.ariba_genes if k in tri_db.keys()])}
        self.ariba_alltrg = sorted(list(set(self.ariba_trg['blm'] + self.ariba_trg['qui'] + 
                                           self.ariba_trg['ami'] + self.ariba_trg['sul'] + 
                                            self.ariba_trg['tri'])))
        
        # Next KmerResistance
        # For this one we also add in a coverage cut-off given our file doesn't seem to be able to do this reliably
        # Plus we are trying to apply the 70% cutoff as it doesn't work easily in the coverage
        # So we will re-apply this.)
        self.kmerres_fl = pd.read_csv(kmerres_files[self.guuid], delimiter = "\t").fillna("")
        self.kmerres_fl = self.kmerres_fl.loc[self.kmerres_fl.template_id > 70.0]
        self.kmerres_genes = sorted(list(set([resfinder_link.loc[k][1] for k in [j for j in list(self.kmerres_fl['#Template']) if "resfindernewid" in j]])))
        self.kmerres_trg = {"blm":sorted([k for k in self.kmerres_genes if k in blm_db.keys()]),
                             "qui":sorted([k for k in self.kmerres_genes if k in qui_db.keys()]), 
                             "ami":sorted([k for k in self.kmerres_genes if k in ami_db.keys()]),
                             "sul":sorted([k for k in self.kmerres_genes if k in sul_db.keys()]), 
                             "tri":sorted([k for k in self.kmerres_genes if k in tri_db.keys()])}
        self.kmerres_alltrg = sorted(list(set(self.kmerres_trg['blm'] + self.kmerres_trg['qui'] + self.kmerres_trg['ami'] + self.kmerres_trg['sul'] + self.kmerres_trg['tri'] )))
        
        # Note for SRST2 we have another bit which doesn't quite work
        # It does not make a file if it finds no genes
        # Therefore we put it into a try except group

        try:
            self.srst2_fl = pd.read_csv(srst2_files[self.guuid], delimiter = "\t").fillna("")
            self.srst2_genes = sorted(list(set([resfinder_link.loc[k][1] for k in list(self.srst2_fl['allele'])])))
        except:
            self.srst2_fl = "N/A"
            self.srst2_genes = []
        self.srst2_trg = {"blm":sorted([k for k in self.srst2_genes if k in blm_db.keys()]),
                             "qui":sorted([k for k in self.srst2_genes if k in qui_db.keys()]), 
                             "ami":sorted([k for k in self.srst2_genes if k in ami_db.keys()]),
                             "sul":sorted([k for k in self.srst2_genes if k in sul_db.keys()]), 
                             "tri":sorted([k for k in self.srst2_genes if k in tri_db.keys()])} 
        self.srst2_alltrg = sorted(list(set(self.srst2_trg['blm'] + self.srst2_trg['qui'] + 
                                           self.srst2_trg['ami'] + self.srst2_trg['sul'] + 
                                            self.srst2_trg['tri'])))
        
        
        ### Aggregating genes. 
        
        self.all_trg = {}
        for k in ["blm", 'qui', 'ami', 'sul', 'tri']:
            self.all_trg[k] = sorted(list(set(self.abricate_trg[k]).union(self.ariba_trg[k]).union(self.kmerres_trg[k]).union(self.srst2_trg[k])))
        self.geno_full = {"abricate":self.abricate_alltrg, "ariba":self.ariba_alltrg, 
                         "kmerres": self.kmerres_alltrg, "srst2":self.srst2_alltrg}
        
        
        
        ### Defining gene families 
        self.gene_families = {}
        for k in self.all_trg.keys():
            k_df = pd.DataFrame(np.zeros((len(self.all_trg[k]),len(self.all_trg[k]) )), 
                               columns = self.all_trg[k], index=self.all_trg[k])
            for l in k_df.index:
                for j in k_df.columns:
                    k_df.loc[l][j] = sim_matrix.loc[resfinder_rlink.loc[l][0]][resfinder_rlink.loc[j][0]]
            self.gene_families[k] = recursive_cluster(k_df, k_df.index)
            
            
        ### Assessing levels of agreement
        # Here we define three things, Firstly, do results agree for all genes for a particular antibiotic class
        # Then do they agree for a whole isolate
        # Then finally we do a bit more delving into the patterns of disagreement
        # Whole isolate level agreement
        
        # Firstly for eac abx class
        self.abx_patterns = {}
        for key in ["blm", 'qui', 'ami', 'sul', 'tri']:
            self.abx_patterns[key] = agreement_pattern(self.abricate_trg[key], 
                                                      self.ariba_trg[key], 
                                                      self.kmerres_trg[key], 
                                                      self.srst2_trg[key])
        self.abx_agreement = {}
        for key in ["blm", 'qui', 'ami', 'sul', 'tri']:
            self.abx_agreement[key] = (self.abx_patterns[key] == [1, 1, 1, 1])

        
        # Then for each whole isolate
        self.full_agreement = (list(set(list(self.abx_agreement.values()))) == [True])
        self.full_patterns = agreement_pattern(self.abricate_alltrg, self.ariba_alltrg, 
                                               self.kmerres_alltrg, self.srst2_alltrg)
        
        
        
        # Note there is a difference between full =  all the patterns in the 5 gene families I have
        # and full across the entiritty of resfinder , this attribute reflects this
        self.wholeresfinder_patterns = agreement_pattern(sorted(self.abricate_genes), sorted(self.ariba_genes),
                                               sorted(self.kmerres_genes), sorted(self.srst2_genes))
        self.wholeresfinder_agreement = (self.wholeresfinder_patterns == [1,1,1,1])


        
        # Lastly for each gene
        self.genes_identified = {}
        self.gene_patterns = {}
        self.gene_group = {}
        for abx in self.gene_families:
            for pat in self.gene_families[abx]:
                pat_id = ":".join(pat)
                pat_string = [pres_bin(pat, self.abricate_alltrg),
                              pres_bin(pat, self.ariba_alltrg), 
                              pres_bin(pat, self.kmerres_alltrg), 
                              pres_bin(pat, self.srst2_alltrg)]
                self.gene_patterns[pat_id] = pres_bin_agreement_pattern(pat_string[0], pat_string[1], pat_string[2], pat_string[3])
                pat_string = "|".join([":".join(i) for i in pat_string])
                self.genes_identified[pat_id] = pat_string
                self.gene_group[pat_id] = abx


# Now with the classes set up we read in everything into an isolates dict
isolates = {}


for run in [str(i) for i in range(1,11)]:
    abricate_dir = "/Users/timdavies/Dropbox_Temp_Outside/pl_comp_data/simulation_run{0}/collated_output/abricate_summary/" .format(run)
    ariba_dir = "/Users/timdavies/Dropbox_Temp_Outside/pl_comp_data/simulation_run{0}/collated_output/ariba_summary/" .format(run)
    kmerres_dir = "/Users/timdavies/Dropbox_Temp_Outside/pl_comp_data/simulation_run{0}/collated_output/kmerres_summary/" .format(run)
    srst2_dir = "/Users/timdavies/Dropbox_Temp_Outside/pl_comp_data/simulation_run{0}/collated_output/srst2_summary/" .format(run)

    ariba_summary = "/Users/timdavies/Dropbox_Temp_Outside/pl_comp_data/simulation_run{0}/collated_output/ariba_summary.csv" .format(run)

    # ABRicate
    abricate_files = [os.path.join(root, f) for root, dirs, files 
                      in os.walk(abricate_dir)
                     for f in files if f != "summary.tab"]
    guuids = [k.split("/")[-1].rstrip("_assem.tab") for k in abricate_files]
    abricate_files = {k.split("/")[-1].rstrip("_assem.tab"):k for k in abricate_files}

    #KmerResistance
    kmerres_files = [os.path.join(root, f) for root, dirs, files in os.walk(kmerres_dir)
                     for f in files if ".KmerRes" in f]
    kmerres_files = {k.split("/")[-1].rstrip(".KmerRes"):k for k in kmerres_files}

    #SRST2
    srst2_files = [os.path.join(root, f) for root, dirs, files in os.walk(srst2_dir) for f in files]
    srst2_files = {k.split("/")[-1].split("_SRST")[0]:k for k in srst2_files}

    #ARIBA
    ariba_names = [k.split("/")[1].split(".")[0] for k in list(pd.read_csv(ariba_summary).name)]
    ariba_summary = pd.read_csv(ariba_summary).fillna("")
    ariba_summary.index = ariba_names
    isolates["run_{0}" .format(run)] = {}
    for n in tnrange(len(guuids)):
        k = guuids[n]
        x = isolate(k)
        isolates["run_{0}" .format(run)][k] = x
        

        
        
        

In [9]:
# Next step lets have a look at the original mutations
simulations_contigs = pd.read_csv("simulation_contigs.csv")
print(len(simulations_contigs))

true_true = 0
true_false = 0
false_false = 0
tt_pats = []
tf_pats =[]
ff_pats = []

with open("interpreting_simulations.csv", "w") as f:
    writer = csv.writer(f, delimiter = ",")
    writer.writerow(["isolate", "pattern", "simulatable", "run_1", "run_2", "run_3", "run_4", "run_5", "run_6", 
                     "run_7", "run_8", 
                     "run_9", "run_10", 
                     "overall"])
    for k in range(len(simulations_contigs)):
        k_data = simulations_contigs.iloc[k]
        k_pattern = k_data.pattern
        k_isolate = k_data.isolate
        k_simulatable = k_data.simulatable
        out_row = [k_isolate, k_pattern,k_simulatable]
        for run in range(1, 11):
            try:
                k_contig = k_data.contig.split("_length_")[0]
                k_key = k_isolate + "_" + k_contig
                if k_pattern in isolates["run_{0}" .format(run)][k_key].genes_identified.keys():
                    true_true += 1
                    out_row.append("True")
                    tt_pats.append(k_pattern)
                else:
                    true_false += 1
                    out_row.append("False")
                    tf_pats.append(k_pattern)
            except:
                false_false += 1
                out_row.append("False")
                ff_pats.append(k_pattern)
        if "True" in out_row[-2:]:
            out_row.append("True")
        else:
            out_row.append("False")
        writer.writerow(out_row)

print(true_true, true_false, false_false)

# for k in Counter(tt_pats):
#     print(Counter(tt_pats)[k], k)
# print('')
# print("")
for k in sorted(Counter(tf_pats).keys(), key = lambda a: Counter(tf_pats)[a], reverse = True):
    print(Counter(tf_pats)[k], k, k in tt_pats)
print("")
print("")




1884
9618 6772 2450
520 qnrS1_1_AB187515:qnrS6_1_HQ631376 False
337 aadA11_2_AJ567827:aadA12_1_AY665771:aadA17_1_FJ460181:aadA1_5_JX185132:aadA24_1_AM711129:ant(3'')-Ia_1_X02340 True
294 sul2_2_AY034138:sul2_9_FJ197818 True
241 aadA11_2_AJ567827:aadA12_1_AY665771:aadA17_1_FJ460181:aadA1_4_JQ480156:aadA24_1_AM711129:ant(3'')-Ia_1_X02340 True
239 dfrA1_10_AF203818:dfrA1_5_EU089668:dfrA1_7_AJ400733:dfrA1_8_X00926:dfrA1_9_AJ238350 True
218 aph(3'')-Ib_2_AF024602:aph(3'')-Ib_3_AF321550 True
209 aph(3'')-Ib_2_AF024602:aph(3'')-Ib_4_AF313472 True
199 dfrA1_10_AF203818:dfrA1_5_EU089668:dfrA1_7_AJ400733:dfrA1_8_X00926 True
198 aadA12_1_AY665771:aadA17_1_FJ460181:aadA1_4_JQ480156:aadA24_1_AM711129:ant(3'')-Ia_1_X02340 True
197 aph(3'')-Ib_2_AF024602:aph(3'')-Ib_5_AF321551 True
190 qnrS1_1_AB187515:qnrS2_1_DQ485530 False
175 aadA11_2_AJ567827:aadA12_1_AY665771:aadA17_1_FJ460181:aadA1_3_JQ414041:aadA24_1_AM711129:ant(3'')-Ia_1_X02340 True
149 aac(3)-IIa_1_X51534:aac(3)-IIe_1_EU022315 True
125 dfrA

In [7]:
print(isolates['59e792c9-73db-426e-9ee0-fc63ccc700af_NODE_61'].genes_identified)

KeyError: '59e792c9-73db-426e-9ee0-fc63ccc700af_NODE_61'